In [8]:
# %pip install tensorflow


In [9]:
# %pip install pandas


In [10]:
# %pip install tqdm
# %pip install matplotlib
# %pip install scikit-learn


In [11]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np


In [12]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'


In [13]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for img in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, img))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

In [14]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [15]:
print(train )

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [16]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [17]:
print(test)
print(test, 'image')

                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]
                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  ima

In [18]:
from tqdm.notebook import tqdm

In [19]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale = True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48,48,1)
    return features
    

In [20]:
# %pip install --upgrade jupyter ipywidgets

In [23]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48,48,1)
    return features

In [ ]:
# train_features is already created and available for use.

  0%|          | 0/28821 [00:00<?, ?it/s]

  0%|          | 0/28821 [00:00<?, ?it/s]

ValueError: cannot reshape array of size 199210752 into shape (28821,48,48,1)

In [30]:
# Only include images with shape (48, 48)
valid_images = []
for image_path in test['image']:
	img = load_img(image_path)
	img_arr = np.array(img)
	if img_arr.shape == (48, 48) or img_arr.shape == (48, 48, 1):
		valid_images.append(image_path)
	else:
		print(f"Skipping {image_path}, shape: {img_arr.shape}")

test_features = extract_features(valid_images)


Skipping images/test\angry\10052.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10065.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10079.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10095.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10121.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10148.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10163.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10215.jpg, shape: (48, 48, 3)
Skipping images/test\angry\1024.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10286.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10315.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10336.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10338.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10371.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10382.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10399.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10415.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10426

0it [00:00, ?it/s]

In [31]:
x_train = train_features/255.0
x_test = test_features/255.0

In [32]:
from sklearn.preprocessing import LabelEncoder

In [33]:
le = LabelEncoder()
le.fit(train['label'])


LabelEncoder()

In [34]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [35]:
y_train = to_categorical(y_train, num_classes= 7)
y_test = to_categorical(y_test, num_classes= 7)

In [36]:
from keras.layers import Dropout
model = Sequential()
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

c:\Users\yuvra\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [41]:
# Rebuild valid_images and y_test to ensure matching cardinality
valid_images = []
valid_labels = []
for image_path, label in zip(test['image'], test['label']):
	img = load_img(image_path)
	img_arr = np.array(img)
	if img_arr.shape == (48, 48) or img_arr.shape == (48, 48, 1):
		valid_images.append(image_path)
		valid_labels.append(label)
	else:
		print(f"Skipping {image_path}, shape: {img_arr.shape}")

test_features = extract_features(valid_images)
x_test = test_features / 255.0

# Encode and one-hot the filtered labels
y_test = le.transform(valid_labels)
y_test = to_categorical(y_test, num_classes=7)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
accuracy = model.evaluate(x_test, y_test)

Skipping images/test\angry\10052.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10065.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10079.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10095.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10121.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10148.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10163.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10215.jpg, shape: (48, 48, 3)
Skipping images/test\angry\1024.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10286.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10315.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10336.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10338.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10371.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10382.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10399.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10415.jpg, shape: (48, 48, 3)
Skipping images/test\angry\10426

0it [00:00, ?it/s]

c:\Users\yuvra\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


ValueError: math domain error

In [42]:
val_loss = model.evaluate(x_test, y_test)
loss = model.evaluate(x_train, y_train)
print("Test loss:",loss[0])
print("Test accuracy:",loss[1])
print('val_loss: ', val_loss[1])



ValueError: math domain error

In [43]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")


In [44]:
from keras.models import model_from_json

In [45]:
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [46]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [47]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [48]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
image = 'images/train/sad/3.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
y_pred = np.argmax(model.predict(x_test), axis=-1)
y_true = np.argmax(y_test, axis=-1)
cm = confusion_matrix(y_true, y_pred)
print("model prediction is ",pred_label)
# Print confusion matrix
print("Confusion Matrix:")
print(cm)
plt.imshow(img.reshape(48,48),cmap='gray')


original image is of sad


TypeError: load_img() got an unexpected keyword argument 'grayscale'

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
image = 'images/train/sad/54.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
# pred_label = label[pred.argmax()]
y_pred = np.argmax(model.predict(x_test), axis=-1)
y_true = np.argmax(y_test, axis=-1)
cm = confusion_matrix(y_true, y_pred)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')
# Assuming you have the test data and labels stored in variables x_test and y_test respectively

print("Accuracy:", accuracy)


In [ ]:
image = 'images/train/fear/2.jpg'
print("original image is of fear")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')
print('Accuracy: ', accuracy[1])

In [ ]:
image = 'images/train/angry/22.jpg'
print("original image is of angry")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')
print('Accuracy: ', accuracy[1])

In [ ]:
image = 'images/train/disgust/864.jpg'
print("original image is of disgust")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')
print('Accuracy: ', accuracy)

In [ ]:
image = 'images/train/happy/7.jpg'
print("original image is of happy")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')
print('Accuracy: ', accuracy)

In [ ]:
image = 'images/test/neutral/44.jpg'
print("original image is of neutral")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')
print('Accuracy: ', accuracy[1])

In [ ]:
image = 'images/test/happy/41.jpg'
print("original image is of happy")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')
print('Accuracy: ', accuracy[1])

In [ ]:
image = 'images/test/fear/101.jpg'
print("original image is of fear")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')
print('Accuracy: ', accuracy[1])

In [ ]:
image = 'images/test/disgust/6926.jpg'
print("original image is of disgust")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')
print('Accuracy: ', accuracy[1])